In [1]:
import re
from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit import Chem, RDLogger
from collections import defaultdict
from hashlib import md5

In [2]:
reagents = {}

In [82]:
def process(smarts):
    """Process a SMARTS reaction string
    into source and target tokens"""
    rxn = AllChem.ReactionFromSmarts(smarts)
    prods = rxn.GetProducts()
    if len(prods) > 1:
        return None

    rxn.Initialize()
    try:
        reactants = list(zip(rxn.GetReactants(), rxn.GetReactingAtoms()))
    except ValueError:
        # Likely that initialization failed
        # print('Failed to initialize')
        return None

    prod_smis = []
    for mol in prods:
        # Clear atom mappings
        [x.ClearProp("molAtomMapNumber") for x in mol.GetAtoms()]
        smi = Chem.MolToSmiles(mol)
        prod_smis.append(smi)

    react_smis = []
    reagent_syms = []
    for mol, atoms in reactants:
        # Clear atom mappings
        [x.ClearProp("molAtomMapNumber") for x in mol.GetAtoms()]

        # Remove molecules with no reacting atoms (reagents)
        # But represent as a symbol
        if not atoms:
            smi = Chem.MolToSmiles(mol)
            if smi not in reagents:
                reagents[smi] = len(reagents)
            reagent_syms.append("[A{}]".format(reagents[smi]))

        else:
            smi = Chem.MolToSmiles(mol)
            react_smis.append(smi)

    source = react_smis
    if reagent_syms:
        source.extend(reagent_syms)
    target = prod_smis

    return source, target


def clean(line):
    return line.strip().split()[0]

In [83]:
# inspect
with open("../data/reactions.rsmi", "r") as f:
    lines = f.readlines()
    for line in lines[:2]:
        print(line.strip())

[C:1]([C:5]1[CH:10]=[CH:9][C:8]([OH:11])=[CH:7][CH:6]=1)([CH3:4])([CH3:3])[CH3:2]>[Ni]>[C:1]([CH:5]1[CH2:6][CH2:7][CH:8]([OH:11])[CH2:9][CH2:10]1)([CH3:4])([CH3:2])[CH3:3]
[Cl-].[Al+3].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][CH2:8][C:9](Cl)=[O:10].[C:12]1([CH:18]([CH3:20])[CH3:19])[CH:17]=[CH:16][CH:15]=[CH:14][CH:13]=1>C(Cl)Cl>[Cl:5][CH2:6][CH2:7][CH2:8][C:9]([C:15]1[CH:16]=[CH:17][C:12]([CH:18]([CH3:20])[CH3:19])=[CH:13][CH:14]=1)=[O:10] |f:0.1.2.3|


In [85]:
seen = set()
reactions = []

limit = 100

with open("../data/reactions.rsmi", "r") as f:
    lines = f.readlines()[:limit]
    it = tqdm(map(process, map(clean, lines)))
    for toks in it:
        if toks is None:
            continue

        # Hash the processed reaction to check for duplicates
        h = md5("_".join("".join(ts) for ts in toks).encode("utf8")).hexdigest()
        if h in seen:
            continue
        else:
            seen.add(h)

        reactions.append(toks)

        it.set_postfix(reactions=len(reactions), reagents=len(reagents))

# Results
print("Reagents:", len(reagents))
print("Reactions:", len(reactions))

# Save
# with open("../data/reagents.dat", "w") as f:
#     lines = []
#     for reagent, id in sorted(reagents.items(), key=lambda kv: kv[1]):
#         lines.append("{}\t{}".format(reagent, id))
#     f.write("\n".join(lines))

# with open("../data/reactions.dat", "w") as f:
#     lines = []
#     for source_toks, target_toks in reactions:
#         lines.append("{}\t{}".format(" ".join(source_toks), " ".join(target_toks)))
#     f.write("\n".join(lines))

0it [00:00, ?it/s, reactions=1, reagents=6][11:47:08] reactant 0 has no mapped atoms.
[11:47:08] reactant 1 has no mapped atoms.
[11:47:08] reactant 2 has no mapped atoms.
[11:47:08] reactant 3 has no mapped atoms.
0it [00:00, ?it/s, reactions=2, reagents=6][11:47:08] reactant 0 has no mapped atoms.
[11:47:08] reactant 1 has no mapped atoms.
[11:47:08] reactant 2 has no mapped atoms.
[11:47:08] reactant 3 has no mapped atoms.
0it [00:00, ?it/s, reactions=3, reagents=6][11:47:08] reactant 2 has no mapped atoms.
0it [00:00, ?it/s, reactions=5, reagents=6][11:47:08] reactant 2 has no mapped atoms.
[11:47:08] reactant 3 has no mapped atoms.
[11:47:08] reactant 4 has no mapped atoms.
[11:47:08] reactant 5 has no mapped atoms.
[11:47:08] reactant 6 has no mapped atoms.
[11:47:08] reactant 7 has no mapped atoms.
0it [00:00, ?it/s, reactions=7, reagents=6][11:47:08] reactant 1 has no mapped atoms.
[11:47:08] reactant 2 has no mapped atoms.
0it [00:00, ?it/s, reactions=8, reagents=6][11:47:08] 

Reagents: 6
Reactions: 70


In [90]:
reagents, product = reactions[1]

In [91]:
reagents

['[Cl-]',
 '[Al+3]',
 '[Cl-]',
 '[Cl-]',
 'O=C(Cl)[CH2][CH2][CH2]Cl',
 '[CH3][CH]([CH3])C1=[CH][CH]=[CH][CH]=[CH]1']

In [92]:
product

['[CH3][CH]([CH3])C1=[CH][CH]=C(C(=O)[CH2][CH2][CH2]Cl)[CH]=[CH]1']